In [1]:
from skyfield.api import load, wgs84

stations_url = 'OneWebDatabase.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

Loaded 636 satellites


Just learning how the package works

In [2]:
by_name = {sat.name: sat for sat in satellites}
print(by_name)

{'ONEWEB-0012': <EarthSatellite ONEWEB-0012 catalog #44057 epoch 2023-07-24 22:03:21 UTC>, 'ONEWEB-0010': <EarthSatellite ONEWEB-0010 catalog #44058 epoch 2023-07-24 22:39:48 UTC>, 'ONEWEB-0008': <EarthSatellite ONEWEB-0008 catalog #44059 epoch 2023-07-25 01:05:45 UTC>, 'ONEWEB-0007': <EarthSatellite ONEWEB-0007 catalog #44060 epoch 2023-07-25 07:19:36 UTC>, 'ONEWEB-0006': <EarthSatellite ONEWEB-0006 catalog #44061 epoch 2023-07-25 07:01:20 UTC>, 'ONEWEB-0011': <EarthSatellite ONEWEB-0011 catalog #44062 epoch 2023-07-25 06:43:07 UTC>, 'ONEWEB-0013': <EarthSatellite ONEWEB-0013 catalog #45131 epoch 2023-07-25 03:27:16 UTC>, 'ONEWEB-0017': <EarthSatellite ONEWEB-0017 catalog #45132 epoch 2023-07-24 21:57:04 UTC>, 'ONEWEB-0020': <EarthSatellite ONEWEB-0020 catalog #45133 epoch 2023-07-25 11:53:13 UTC>, 'ONEWEB-0021': <EarthSatellite ONEWEB-0021 catalog #45134 epoch 2023-07-25 12:13:50 UTC>, 'ONEWEB-0022': <EarthSatellite ONEWEB-0022 catalog #45135 epoch 2023-07-25 12:49:14 UTC>, 'ONEWEB-0

In [3]:
ts = load.timescale()
t0 = ts.utc(2023, 7, 24, 0, 0, 0)
t1 = ts.utc(2023, 7, 25)

place = wgs84.latlon(51.5072, -0.1276)
t, events = satellites[1].find_events(place, t0, t1, altitude_degrees=30.0)
event_names = 'rise above 30°', 'culminate', 'set below 30°'
for ti, event in zip(t, events):
    name = event_names[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)
    print(place.at(ti).position.km)

#print(place.at(t))

2023 Jul 24 09:51:17 rise above 30°
[ 6.82430408e+01  6.37777190e+03 -3.58564666e-01]
2023 Jul 24 09:53:38 culminate
[ 2.55698784e+00  6.37813648e+03 -2.09145919e-01]
2023 Jul 24 09:55:59 set below 30°
[-6.31541478e+01  6.37782433e+03 -5.96485114e-02]
2023 Jul 24 21:42:00 rise above 30°
[-2.72535095e+02 -6.37231163e+03  8.23104230e-01]
2023 Jul 24 21:44:54 culminate
[-1.91524542e+02 -6.37526074e+03  6.38900459e-01]
2023 Jul 24 21:47:49 set below 30°
[-1.10087455e+02 -6.37718685e+03  4.53693677e-01]


In [4]:
import VectorClass as vc

xyz = [sat.at(t0).position.km for sat in satellites]
place_L = wgs84.latlon(51.5072, -0.1276)
place_B = wgs84.latlon(52.4862, -1.8904)
dist = [{'London': (sat - place_L).at(t0).position.km, 'Birmingham': (sat - place_B).at(t0).position.km} for sat in satellites]

vectors = {satellites[i].name: [xyz[i], dist[i]] for i in range(len(satellites))}
print(vectors)
print(vectors['ONEWEB-0034'][-1]['London'])
#I don't understand this way of storing

{'ONEWEB-0012': [array([ 1310.50808529,  6199.73781244, -4163.77002458]), {'London': array([ -751.64685427,  9608.19257808, -9128.06145028]), 'Birmingham': array([ -604.00624654,  9594.7913684 , -9195.46558595])}], 'ONEWEB-0010': [array([-1133.48759944, -6666.72362266, -3428.17203794]), {'London': array([-3195.642539  , -3258.26885702, -8392.46346364]), 'Birmingham': array([-3048.00193127, -3271.67006669, -8459.86759931])}], 'ONEWEB-0008': [array([-176.75629894,  466.06891753, 7550.46645314]), {'London': array([-2238.9112385 ,  3874.52368316,  2586.17502745]), 'Birmingham': array([-2091.27063077,  3861.12247349,  2518.77089178])}], 'ONEWEB-0007': [array([-1149.38566329,  3847.55305118, -6438.25642559]), {'London': array([ -3211.54060285,   7256.00781682, -11402.54785129]), 'Birmingham': array([ -3063.89999512,   7242.60660714, -11469.95198696])}], 'ONEWEB-0006': [array([-2575.98514009,  7127.34295467,   244.13551482]), {'London': array([-4638.14007965, 10535.79772031, -4720.15591087]),

In [6]:
Visibility = {
    'London': [vc.Vector(place_L.at(t0).position.km[0], place_L.at(t0).position.km[1], place_L.at(t0).position.km[2], 'cartesian'), 
               {sat.name: [vc.Vector((sat - place_L).at(t0).position.km[0], (sat - place_L).at(t0).position.km[1], (sat - place_L).at(t0).position.km[2], 'cartesian'), 
                           (sat - place_L).at(t0).altaz()[0].degrees] for sat in satellites if (sat - place_L).at(t0).altaz()[0].degrees > 25}],

    'Birmingham': [vc.Vector(place_B.at(t0).position.km[0], place_B.at(t0).position.km[1], place_B.at(t0).position.km[2], 'cartesian'), 
                   {sat.name: [vc.Vector((sat - place_B).at(t0).position.km[0], (sat - place_B).at(t0).position.km[1], (sat - place_B).at(t0).position.km[2], 'cartesian'), 
                               (sat - place_B).at(t0).altaz()[0].degrees] for sat in satellites if (sat - place_B).at(t0).altaz()[0].degrees > 25}]
            }

print(Visibility)
print(Visibility['London'][-1])
print(Visibility['London'][-1]['ONEWEB-0247'][0].magnitude())

{'London': [<VectorClass.Vector object at 0x00000229239E3688>, {'ONEWEB-0184': [<VectorClass.Vector object at 0x00000229239E37C8>, 37.97251679221645], 'ONEWEB-0198': [<VectorClass.Vector object at 0x00000229239E3808>, 32.52850677971246], 'ONEWEB-0247': [<VectorClass.Vector object at 0x00000229239E3F88>, 41.51187071337718], 'ONEWEB-0228': [<VectorClass.Vector object at 0x00000229239E4988>, 27.763359030453273], 'ONEWEB-0234': [<VectorClass.Vector object at 0x00000229239E4AC8>, 37.59859435859155], 'ONEWEB-0321': [<VectorClass.Vector object at 0x00000229239E4B48>, 27.45204754808131], 'ONEWEB-0357': [<VectorClass.Vector object at 0x00000229239E4C08>, 52.16105875999905], 'ONEWEB-0364': [<VectorClass.Vector object at 0x00000229239E4BC8>, 66.57203482058384], 'ONEWEB-0674': [<VectorClass.Vector object at 0x00000229239E4CC8>, 26.101604766057466]}], 'Birmingham': [<VectorClass.Vector object at 0x00000229239E4D08>, {'ONEWEB-0184': [<VectorClass.Vector object at 0x00000229239E4048>, 36.534686157201

In [7]:
Visibility['Text'] = 0

In [10]:
cities={'London':[51.5072, -0.1276],'Birmingham':[52.4862, -1.8904]}

Locations = {}
for city,latlon in zip(cities.keys(),cities.values()):
    Locations[city]=wgs84.latlon(*latlon)

print(Locations)

Visibility = {}
for city, place in zip(Locations.keys(), Locations.values()):
    Visibility[city] = [vc.Vector(*(place.at(t0).position.km), 'cartesian'), 
                        {sat.name: [vc.Vector(*((sat - place).at(t0).position.km), 'cartesian'), 
                                    (sat - place).at(t0).altaz()[0].degrees, vc.Vector(*(sat.at(t0).position.km), 'cartesian')] for sat in satellites if (sat - place).at(t0).altaz()[0].degrees > 25}
                        ]

print(Visibility)
print(Visibility['London'][0].cartesian())

{'London': <GeographicPosition WGS84 latitude +51.5072 N longitude -0.1276 E elevation 0.0 m>, 'Birmingham': <GeographicPosition WGS84 latitude +52.4862 N longitude -1.8904 E elevation 0.0 m>}
{'London': [<VectorClass.Vector object at 0x000001E7D5CEB708>, {'ONEWEB-0184': [<VectorClass.Vector object at 0x000001E7C4187848>, 37.97251679221645, <VectorClass.Vector object at 0x000001E7D5CEBAC8>], 'ONEWEB-0198': [<VectorClass.Vector object at 0x000001E7D5CEB888>, 32.52850677971246, <VectorClass.Vector object at 0x000001E7C2713848>], 'ONEWEB-0247': [<VectorClass.Vector object at 0x000001E7C2713A08>, 41.51187071337718, <VectorClass.Vector object at 0x000001E7C2713A48>], 'ONEWEB-0228': [<VectorClass.Vector object at 0x000001E7C27138C8>, 27.763359030453273, <VectorClass.Vector object at 0x000001E7C27137C8>], 'ONEWEB-0234': [<VectorClass.Vector object at 0x000001E7D5CDE888>, 37.59859435859155, <VectorClass.Vector object at 0x000001E7D5CDE3C8>], 'ONEWEB-0321': [<VectorClass.Vector object at 0x0000

In [16]:
import itur
import astropy.units as u

# Ground station coordinates (Boston)
lat_GS = 47
lon_GS = 0

h_sat=35786e3
lat_sat=0
lon_sat=0

el = itur.utils.elevation_angle(h_sat, lat_sat, lon_sat, lat_GS, lon_GS) #Not necessary as I have already calculated it
f =  12* u.GHz    # Link frequency
p = 0.5 #percentage of time the value is exceeded
Ag, Ac, Ar, As, A  =itur.atmospheric_attenuation_slant_path(lat_GS, lon_GS,
                                                            f, el, p, D=5 * u.m,
                                                            eta=0.65, return_contributions=True)
print(Ag, Ac, Ar, As, A,A-Ar )

0.1307592281764672 dB 0.26102538774922807 dB 0.4471011554227243 dB 0.15737166633928917 dB 0.8561619042888893 dB 0.409060748866165 dB


Integration planning!

The entire code needs to run over time. This means at each timestep, the position of a Terminal and Satellite, the Visibility, the Link Budget and finally the SNR need to be calculated.

This means the Positions of the Satellite instances and Terminals need to be able to update with time.

Before time sim, a dataset of all orbiting satellites, a dataset of all investigated Terminals, maybe hubs should be initalised
The best approach might consist of having a extra attribute, which contians special functions used to update the attributes I coded.
Start drafting in a new folder again as it will get quite complicated.

Instead it might be smarter to store some kind of table or another array that has a initalised class for each satellite in the skyfied array.

In [18]:
import pickle #Prob not needed!

with open('file.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(Visibility, file)

In [20]:
with open('file.pkl', 'rb') as file:
    # A new file will be created
    test = pickle.load(file)

test

{'London': [<VectorClass.Vector at 0x1e7fd63ebc8>,
  {'ONEWEB-0184': [<VectorClass.Vector at 0x1e7fd639108>,
    37.97251679221645,
   'ONEWEB-0198': [<VectorClass.Vector at 0x1e7fd639fc8>,
    32.52850677971246,
   'ONEWEB-0247': [<VectorClass.Vector at 0x1e7fd6392c8>,
    41.51187071337718,
   'ONEWEB-0228': [<VectorClass.Vector at 0x1e7fd639248>,
    27.763359030453273,
   'ONEWEB-0234': [<VectorClass.Vector at 0x1e7fd6393c8>,
    37.59859435859155,
   'ONEWEB-0321': [<VectorClass.Vector at 0x1e7fd639948>,
    27.45204754808131,
   'ONEWEB-0357': [<VectorClass.Vector at 0x1e7fd639608>,
    52.16105875999905,
   'ONEWEB-0364': [<VectorClass.Vector at 0x1e7e6aedc08>,
    66.57203482058384,
   'ONEWEB-0674': [<VectorClass.Vector at 0x1e7e6aedb48>,
    26.101604766057466,
    <VectorClass.Vector at 0x1e7e6aed288>]}],
 'Birmingham': [<VectorClass.Vector at 0x1e7fd631dc8>,
  {'ONEWEB-0184': [<VectorClass.Vector at 0x1e7fd631248>,
    36.53468615720184,
   'ONEWEB-0198': [<VectorClass.Vect